In [4]:
import pandas as pd
import math
import os
from tqdm import tqdm

# ฟังก์ชัน Haversine เพื่อคำนวณระยะห่างระหว่างพิกัดสองจุด
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # รัศมีของโลกในกิโลเมตร
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # ระยะห่างเป็นกิโลเมตร
    return distance

# ฟังก์ชันค้นหาชื่อจังหวัดจากพิกัด (ใช้ระยะห่าง)
def get_province_name(lat, lon, province_dict):
    closest_province = None
    min_distance = float('inf')  # เริ่มต้นด้วยระยะห่างที่ไม่จำกัด
    for (province_lat, province_lon), province_name in province_dict.items():
        distance = haversine(lat, lon, province_lat, province_lon)
        if distance < min_distance:
            min_distance = distance
            closest_province = province_name
    if closest_province:
        return closest_province
    else:
        return "Unknown Province"  # ถ้าไม่พบจังหวัดให้คืนค่า Unknown Province

# ฟังก์ชันการประมวลผลข้อมูลในแต่ละ chunk
def process_chunk(chunk, province_dict):
    chunk['Province_en'] = [get_province_name(lat, lon, province_dict) for lat, lon in zip(chunk['Latitude'], chunk['Longitude'])]
    chunk = chunk.drop(columns=['Latitude', 'Longitude'])
    return chunk

# อ่านข้อมูลจังหวัด
province_file = "/Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Provice data/Thailand Provinces.csv"
province_df = pd.read_csv(province_file)

# สร้าง dictionary สำหรับจังหวัด (ใช้พิกัดละติจุดและลองติจุดเป็น key)
province_dict = {
    (row['province_lat'], row['province_lon']): row['province_name']
    for _, row in province_df.iterrows()
}

# ฟังก์ชันหลักที่ประมวลผลไฟล์หลาย ๆ ไฟล์
def process_multiple_files(input_dir, output_dir, province_dict):
    # ลูปผ่านไฟล์ทั้งหมดในโฟลเดอร์
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):
            input_file_path = os.path.join(input_dir, file_name)
            output_file_path = os.path.join(output_dir, f"processed_{file_name}")

            # อ่านไฟล์ข้อมูลที่มีพิกัด Latitude และ Longitude
            df = pd.read_csv(input_file_path)

            # ตรวจสอบว่ามีคอลัมน์ Latitude และ Longitude หรือไม่
            if 'Latitude' in df.columns and 'Longitude' in df.columns:
                # แสดง progress bar พร้อมชื่อไฟล์
                print(f"\nProcessing file: {file_name}")
                with tqdm(total=len(df), desc=f"Processing {file_name}", unit='row') as pbar:
                    df = process_chunk(df, province_dict)
                    pbar.update(len(df))  # Update progress bar after processing the chunk

                # บันทึกผลลัพธ์ลงไฟล์ CSV
                df.to_csv(output_file_path, index=False)
                print(f"Processed data saved to {output_file_path}")
            else:
                print(f"⚠️ Missing Latitude or Longitude columns in {file_name}")
        else:
            print(f"⚠️ Skipping non-CSV file: {file_name}")

# กำหนดที่อยู่ของโฟลเดอร์ข้อมูล
input_dir = "/Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Selected Month/3_10_11_Data"
output_dir = "/Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/data_with_province"
 
# ประมวลผลไฟล์ทั้งหมดในโฟลเดอร์
process_multiple_files(input_dir, output_dir, province_dict)


Processing file: 20241102.csv


Processing 20241102.csv: 100%|██████████| 804726/804726 [01:11<00:00, 11192.73row/s]


Processed data saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/data_with_province/processed_20241102.csv

Processing file: 20241116.csv


Processing 20241116.csv: 100%|██████████| 786512/786512 [01:06<00:00, 11903.86row/s]


Processed data saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/data_with_province/processed_20241116.csv

Processing file: 20230328.csv


...

Processed data saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/data_with_province/processed_20230325.csv

Processing file: 20230331.csv


Processing 20230331.csv: 100%|██████████| 1012998/1012998 [00:37<00:00, 26881.62row/s]


Processed data saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/data_with_province/processed_20230331.csv

Processing file: 20241127.csv


Processing 20241127.csv: 100%|██████████| 795982/795982 [00:29<00:00, 26820.84row/s]


Processed data saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/data_with_province/processed_20241127.csv


In [5]:
import pandas as pd
import os

# รายชื่อจังหวัดที่ต้องการ
selected_provinces = [
    "กรุงเทพมหานคร", "นนทบุรี", "ปทุมธานี", "สมุทรปราการ", "สมุทรสาคร", "นครปฐม"
]

# ฟังก์ชันเลือกแค่จังหวัดที่ต้องการและบันทึกไฟล์ใหม่
def select_and_save_provinces(input_dir, output_dir, selected_provinces):
    # ลูปผ่านไฟล์ทั้งหมดในโฟลเดอร์
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):
            input_file_path = os.path.join(input_dir, file_name)
            output_file_path = os.path.join(output_dir, f"selected_{file_name}")

            # อ่านไฟล์ข้อมูล
            df = pd.read_csv(input_file_path)

            # กรองเฉพาะจังหวัดที่ต้องการ
            filtered_df = df[df['Province_en'].isin(selected_provinces)]

            # บันทึกผลลัพธ์ลงไฟล์ใหม่
            filtered_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
            print(f"✅ Processed {file_name} and saved to {output_file_path}")
        else:
            print(f"⚠️ Skipping non-CSV file: {file_name}")

# กำหนดที่อยู่ของโฟลเดอร์ข้อมูล
input_dir = "/Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/data_with_province"
output_dir = "/Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province"

# เรียกใช้ฟังก์ชัน
select_and_save_provinces(input_dir, output_dir, selected_provinces)

✅ Processed processed_20230305.csv and saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province/selected_processed_20230305.csv
✅ Processed processed_20230311.csv and saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province/selected_processed_20230311.csv
✅ Processed processed_20231027.csv and saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province/selected_processed_20231027.csv
✅ Processed processed_20220314.csv and saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province/selected_processed_20220314.csv
✅ Processed processed_20220328.csv and saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province/selected_processed_20220328.csv
✅ Processed processed_20221022.csv and saved to /Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province/selected_processed

In [6]:
import pandas as pd
import os

# แมปชื่อจังหวัดภาษาไทยไปเป็นภาษาอังกฤษ
province_translation = {
    "กรุงเทพมหานคร": "Bangkok",
    "นนทบุรี": "Nonthaburi",
    "ปทุมธานี": "Pathum Thani",
    "สมุทรปราการ": "Samut Prakan",
    "สมุทรสาคร": "Samut Sakhon",
    "นครปฐม": "Nakhon Pathom"
}

# ฟังก์ชันสำหรับการเปลี่ยนชื่อจังหวัดในไฟล์
def translate_provinces_in_file(input_dir, province_translation):
    # ลูปผ่านไฟล์ทั้งหมดในโฟลเดอร์
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):
            input_file_path = os.path.join(input_dir, file_name)

            # อ่านไฟล์ข้อมูล
            df = pd.read_csv(input_file_path)

            # เปลี่ยนชื่อจังหวัดจากภาษาไทยเป็นภาษาอังกฤษ
            df['Province_en'] = df['Province_en'].replace(province_translation)

            # บันทึกไฟล์ใหม่ลงในไฟล์เดิม
            df.to_csv(input_file_path, index=False, encoding='utf-8-sig')
            print(f"✅ Processed {file_name} and saved the updated file.")
        else:
            print(f"⚠️ Skipping non-CSV file: {file_name}")

# กำหนดที่อยู่ของโฟลเดอร์ที่มีไฟล์ .csv
input_dir = "/Users/kukkaii/Documents/DA PRoject/Clean_RoadData/Connect the provice/Selected_Province"

# เรียกใช้ฟังก์ชัน
translate_provinces_in_file(input_dir, province_translation)

✅ Processed selected_processed_20221002.csv and saved the updated file.
✅ Processed selected_processed_20221016.csv and saved the updated file.
✅ Processed selected_processed_20220308.csv and saved the updated file.
✅ Processed selected_processed_20220320.csv and saved the updated file.
✅ Processed selected_processed_20231013.csv and saved the updated file.
✅ Processed selected_processed_20230319.csv and saved the updated file.
✅ Processed selected_processed_20231007.csv and saved the updated file.
✅ Processed selected_processed_20230331.csv and saved the updated file.
✅ Processed selected_processed_20230325.csv and saved the updated file.
✅ Processed selected_processed_20241127.csv and saved the updated file.
✅ Processed selected_processed_20241126.csv and saved the updated file.
✅ Processed selected_processed_20230324.csv and saved the updated file.
✅ Processed selected_processed_20230330.csv and saved the updated file.
✅ Processed selected_processed_20231006.csv and saved the update